<a href="https://colab.research.google.com/github/AhmedAl-Hayali/HAB_Detection_in_Lakes/blob/main/2PX3_Final_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A lot of the code is self-authored and adapted from previous independent projects hence I have so few references. If anything, I may have modified some code from Eli Stevens, Luca Antiga, and Thomas Viehmann's "Deep Learning with PyTorch" in previous iterations of the code used here.

In [ ]:
# Imports & Formatting
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)

import cv2
import os

from PIL import Image, ImageEnhance
import requests
from io import BytesIO

from torchvision import transforms

import datetime

In [ ]:
# Importing training data
from zipfile import ZipFile
file_name = "2px3_wk7_imgs.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
# Setting all operations to operate on the GPU for better performance
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [ ]:
# Extracting training image data; credit: 2PX3 Wk 5 code

labels = ['alge', 'no_alge']
img_size = 224

def get_data():
    # Pre-instantiate empty lists to store data and corresponding labels
    data = [] 
    data_labels = []

    # Iterating over labels
    for label in labels: 
        path = label + "_imgs"
        class_num = labels.index(label)
        # Iterating over images for each label
        for img in os.listdir(path):
            try:
                # Reading and storing images in an np array
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append(resized_arr)
                data_labels.append(class_num)
            except Exception as e:
                print(e)
        x = np.array(data)
        y = np.array(data_labels)
    return x, y

In [ ]:
# Storing images and corresponding labels in np arrays
data, labs = get_data()

In [ ]:
# Preprocessing training data
data_mod = torch.tensor(data, dtype=torch.float32).permute(0, 3, 1, 2)

# Computing mean and standard deviation
data_sd, data_mean = torch.std_mean(data_mod, dim=(0,2, 3)) # Collapse spatial and batch dims
# tensor([72.3099, 65.2078, 67.4276]) tensor([112.5567, 120.7564, 108.9186])

# Normalizing data
data_normal = transforms.functional.normalize(data_mod, mean=data_mean, std=data_sd)

In [ ]:
# Extracting and preprocessing validation data
def get_img(link):
    try:
        img_html = requests.get(link)
        img = Image.open(BytesIO(img_html.content))
    except Exception as e:
        print(e)
    
    return img

preprocess = transforms.Compose([
                                 transforms.CenterCrop([224, 224]), # scale input to 224 x 224
                                 transforms.ToTensor(), # transform to a tensor, 3D array w/ color & 2 spatial dims
                                 transforms.Normalize([112.5567, 120.7564, 108.9186], [72.3099, 65.2078, 67.4276])
                                 ])

# Links for images of algal-bloom water and water without an algal bloom
alge_tests_links = ["https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200131-05-135x104.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200131-07-135x114.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200131-08-135x90.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200131-04-135x108.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20210712-1-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200131-09-135x127.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20210712-4-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20210712-4-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20210712-5-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200131-03-135x90.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20210712-2-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200727-01-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200727-03-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20210712-6-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20210712-7-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200131-01-135x91.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200727-04-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200727-02-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20200131-02-135x90.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20210712-9-thumbnail.jpg", "https://www.canada.ca/content/dam/eccc/images/eolakewatch/20210712-3-thumbnail.jpg"]

noalge_tests_links = ["https://www.nasa.gov/sites/default/files/styles/full_width_feature/public/iss036e035632.jpg", "https://images.fineartamerica.com/images-medium-large-5/1-satellite-view-of-crater-lake-oregon-panoramic-images.jpg", "https://content.satimagingcorp.com/static/galleryimages/ikonos-atsukeshi-lake-japan.jpg", "https://www.overv.eu/wp-content/uploads/2017/11/lake-victoria.jpg", "https://eoportal.org/documents/163813/248818/Lake-st-Clair-Deimos-1", "https://www.fws.gov/sites/default/files/styles/scale_width_1200/public/2020-10/LakeOntario_NOAA_publicdomain.jpg?itok=UbAMoCle", "https://static.eos.com/wp-content/uploads/2021/11/Lake-Ontario.jpg", "https://www.robertharding.com/watermark.php?type=preview&im=RM/RH/HORIZONTAL/1348-3045", "https://live.staticflickr.com/7544/15869976110_4c7ee8da95_b.jpg"]

In [ ]:
# Putting the data into a list of tuples, each tuple containing the tensor of the image and its corresponding label (0 alge, 1 no alge)

# Training data
data_norm_lab = [(d, torch.tensor(labs[i], dtype=torch.float)) for i, d in enumerate(data_normal)]

# Validation data; This will take some time because it gets the image from the internet so we deal with data traffic delay :/
val_data = [(preprocess(get_img(l)), 0) for l in alge_tests_links] + [(preprocess(get_img(l)), 1) for l in noalge_tests_links]

In [ ]:
# Training data loader (for batching 16 images at a time); Comment out when not training
train_loader = torch.utils.data.DataLoader(data_norm_lab, batch_size=16,
                                           shuffle=True)

# Validation data loader (for batching 16 images at a time); Comment out when not validating
val_loader = torch.utils.data.DataLoader(val_data, batch_size=16,
                                           shuffle=True)

# Function to validate results after training
def validate(model, train_loader, val_loader):
    # Instantiating empty dict for separating train and val accuracies
    accdict = {}
    # Iterating through loaders (train & val)
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        # Correct & total predictions tally
        correct = 0
        total = 0

        # Operating without gradients because this is evaluation not training
        with torch.no_grad():
            # Iterating through images and corresponding labels in current loader
            for imgs, labels in loader:
                # Ensuring images and labels are on the same device (cpu or gpu)
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)

                # Model output for proability of image being each class
                outputs = model(imgs)
                # print(outputs)
                # Highest-probability class
                _, predicted = torch.max(outputs, dim=-1)

                # Updating total & correct tallies
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
        
        # Outputting accuracy for specified loader
        print("Accuracy {}: {:.2f}".format(name , correct / total))
        # Caching accuracy result for specified loader in dict
        accdict[name] = correct / total
    return accdict

In [ ]:
def training_loop_l2reg(n_epochs, optimizer, model, loss_fn,
                        train_loader):
    # Traversing dataset for n_epochs times and updating parameters as necessary.
    for epoch in range(1, n_epochs + 1):
        # Tracking (train) loss across individual epochs
        loss_train = 0.0
        # Traversing images and corresponding labels in train data loader
        for imgs, labls in train_loader:
            imgs = imgs.to(device=device)
            labls = labls.to(device=device, dtype=torch.long)

            # Model predictions for each image
            outputs = model(torch.squeeze(imgs))
            # print(outputs)
            
            # Computing loss and regularizing (l2 reg) it
            loss = loss_fn(outputs, labls)
            l2_lambda = 0.001
            l2_norm = sum(p.pow(2.0).sum()
                          for p in model.parameters())
            loss = loss + l2_lambda * l2_norm

            # Backpropagating after clearing gradient to prevent gradient accumulation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Updating loss
            loss_train += loss.item()
        # Updating client every 10 epochs of current epoch and training loss
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))

In [ ]:
# Using a 7x7 kernel and 3x3 padding (0-padding to be specific)
k_size = (7, 7)
pad = (3, 3)

class NetRes(nn.Module):
    # Net architecture
    def __init__(self, n_chans1=32, img_size=224):
        super().__init__()
        self.n_chans1 = n_chans1
        self.img_size = img_size
        # 1st conv layer - C : 3 -> n_chans1, K: 7x7 kernel, P: 3x3 0-padding
        self.conv1 = nn.Conv2d(3, n_chans1, kernel_size=k_size, padding=pad)
        # 2nd conv layer - C : n_chans1 -> n_chans1 // 2, K: 7x7 kernel, P: 3x3
        # 0-padding
        self.conv2 = nn.Conv2d(n_chans1, n_chans1 // 2, kernel_size=k_size,
                               padding=pad)
        # 1st conv layer - C : n_chans1 // 2 -> n_chans1 // 2, K: 7x7 kernel,
        # P: 3x3 0-padding
        self.conv3 = nn.Conv2d(n_chans1 // 2, n_chans1 // 2,
                               kernel_size=k_size, padding=pad)
        # 1st fully conn layer - collapse from (img_size/8)^2 * n_chans1 // 2
        # to 32
        self.fc1 = nn.Linear((self.img_size >> 3)**2 * self.n_chans1 // 2, 32)
        # 2nd fully conn layer - collapse from 32 to 2
        self.fc2 = nn.Linear(32, 2)
        
    def forward(self, x):
        # Forward pass - pooling after every conv layer, with a residual at
        # the 3rd
        out = F.max_pool2d(torch.relu(self.conv1(x)), 2)
        out = F.max_pool2d(torch.relu(self.conv2(out)), 2)
        out1 = out
        out = F.max_pool2d(torch.relu(self.conv3(out)) + out1, 2)
        # Reshaping from NxCxHxW tensor to Nx(C*H*W) matrix for dimension
        # compatibility
        out = out.view(-1, (self.img_size >> 3)**2 * self.n_chans1 // 2)
        # Relu after first fully conn layer
        out = torch.relu(self.fc1(out))
        # Relu after 2nd fully conn layer + reshaping for dimension
        # compatibility
        out = torch.relu(torch.squeeze(self.fc2(out)))
        return out

# Instantiating model to the net architecture above, and setting device
model = NetRes(n_chans1=16).to(device=device)
# Setting optimizer to AdamW (refer to https://arxiv.org/pdf/1711.05101.pdf for extreme detail; I just use it because it's advised)
optimizer = optim.AdamW(model.parameters(), lr=1e-2)
# Binary cross entropy loss - there are better options, but it doesn't matter for this project
loss_fn = nn.CrossEntropyLoss()

In [ ]:
# Calling training loop to train model; Comment out when not training
model.train()
training_loop_l2reg(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2022-04-14 22:10:51.117251 Epoch 1, Training loss 3.0362828969955444
2022-04-14 22:10:53.201709 Epoch 10, Training loss 2.7107034623622894
2022-04-14 22:10:55.514215 Epoch 20, Training loss 2.423653483390808
2022-04-14 22:10:57.833788 Epoch 30, Training loss 2.1904627084732056
2022-04-14 22:11:00.164415 Epoch 40, Training loss 1.9970385432243347
2022-04-14 22:11:02.489525 Epoch 50, Training loss 1.8345318585634232
2022-04-14 22:11:04.824346 Epoch 60, Training loss 1.6968109160661697
2022-04-14 22:11:07.151945 Epoch 70, Training loss 1.579355463385582
2022-04-14 22:11:09.477256 Epoch 80, Training loss 1.4786954522132874
2022-04-14 22:11:11.800156 Epoch 90, Training loss 1.392090991139412
2022-04-14 22:11:14.128669 Epoch 100, Training loss 1.3173342943191528


In [ ]:
# Saving model after training; comment out if not training
# torch.save(model.state_dict(), f="2px3_final_model")

# Loading model we saved earlier
model.load_state_dict(torch.load("/content/2px3_final_model"))
# Setting model to eval mode (refer to https://stackoverflow.com/a/60018731 for details on why we do this)
model.eval()

NetRes(
  (conv1): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv2): Conv2d(16, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv3): Conv2d(8, 8, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (fc1): Linear(in_features=6272, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)

In [ ]:
# Transferring model to GPU then validating
model.cuda()
validate(model, train_loader, val_loader)

Accuracy train: 0.99
Accuracy val: 0.70


{'train': 0.9919354838709677, 'val': 0.7}